In [1]:
import pandas as pd

In [2]:
# Read the CSV files
cal = pd.read_csv('data/calendar2024.csv')
lis = pd.read_csv('data/listings2024.csv') 
rev = pd.read_csv('data/reviews2024.csv')

In [3]:
cal.drop(columns=['adjusted_price'], inplace=True)

null_cols_lis = lis.columns[lis.isna().all()].tolist()
lis = lis.drop(columns=null_cols_lis)

lis.drop(columns=['scrape_id'], inplace=True)

rev.drop(columns=['reviewer_name'], inplace=True)

In [ ]:
cal['price'] = cal['price'].str.replace(r'[\$,]', '', regex=True)

cal = cal.rename(columns={'price': 'price($)'})
cal['price($)'] = pd.to_numeric(cal['price($)'], errors='coerce')

cal['available'] = cal['available'] == 't'
lis['instant_bookable'] = lis['instant_bookable'] == 't'

cal['date'] = pd.to_datetime(cal['date'])
rev['date'] = pd.to_datetime(rev['date'])
lis['last_scraped'] = pd.to_datetime(lis['last_scraped'])
lis['host_since'] = pd.to_datetime(lis['host_since'])

rev['comments'].astype("string")#(str)

lis.info()